# Tutoriel LangChain avec Ollama


Tutoriel : [LangChain for LLM application developpment](https://www.deeplearning.ai/short-courses/langchain-for-llm-application-development/) de DeepLearning.ai.  


### SOMMAIRE  
[1. Models, prompts and parsers](#models-prompts-and-parsers)  
[2. Memory](#memory)  
[3. Chains](#chains)  
[4. Question and answer](#question-and-answer)  
[5. Evaluation](#evaluation)  
[6. Agents](#agents)  

##### Packages requirements list

In [2]:
!pip freeze

aiohappyeyeballs==2.4.0
aiohttp==3.10.5
aiosignal==1.3.1
annotated-types==0.7.0
anyio==4.4.0
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
arrow==1.3.0
asttokens==2.4.1
async-lru==2.0.4
attrs==24.2.0
babel==2.16.0
beautifulsoup4==4.12.3
bleach==6.1.0
certifi==2024.8.30
cffi==1.17.1
charset-normalizer==3.3.2
click==8.1.7
colorama==0.4.6
comm==0.2.2
dataclasses-json==0.6.7
debugpy==1.8.5
decorator==5.1.1
defusedxml==0.7.1
dnspython==2.6.1
docarray==0.40.0
email_validator==2.2.0
executing==2.1.0
fastapi==0.115.0
fastapi-cli==0.0.5
fastjsonschema==2.20.0
filelock==3.16.0
fqdn==1.5.1
frozenlist==1.4.1
fsspec==2024.9.0
greenlet==3.0.3
h11==0.14.0
httpcore==1.0.5
httptools==0.6.1
httpx==0.27.2
huggingface-hub==0.24.7
idna==3.8
ipykernel==6.29.5
ipython==8.27.0
ipywidgets==8.1.5
isoduration==20.11.0
jedi==0.19.1
Jinja2==3.1.4
joblib==1.4.2
json5==0.9.25
jsonpatch==1.33
jsonpointer==3.0.0
jsonschema==4.23.0
jsonschema-specifications==2023.12.1
jupyter==1.1.1
jupyter-console==6.6.3
jupyter-ev

### <a id='models-prompts-and-parsers'>1. Models, prompts and parsers</a>

In [3]:
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import PromptTemplate

In [4]:
model_name = 'gemma2:2b-instruct-q4_1'

In [5]:
# Hello, world (with LangChain and gemma2)

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate.from_template(template)


llm = OllamaLLM(model=model_name)
chain = prompt | llm

question = "What NFL team won the Super Bowl in the year Justin Bieber was born?"
chain.invoke({"question": question})

"Here's how to solve this:\n\n1. **Find out when Justin Bieber was born:** Justin Bieber was born on March 1, 1994.\n2. **Find the Super Bowl winner for that year:** The Super Bowl in the year Justin Bieber was born (Super Bowl XXVIII) took place in January 1994.\n3. **Determine the winning team:** The Atlanta Falcons won Super Bowl XXVII.  \n\nLet me know if you have any more questions! 🏈🏆  \n"

In [6]:
# Simple test with the ollama python library directly
import ollama

def get_completion(prompt, model=model_name):
    messages = [{'role': 'user', 'content': prompt}]
    response = ollama.chat(model=model, messages=messages)

    return response['message']['content']

In [7]:
get_completion('What is 1+1?')

'1 + 1 = **2** \n'

In [8]:
# pirate english
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

style = """American English \
in a calm and respectful tone
"""

In [9]:
prompt = f"""Translate the text that is delimited by triple backticks into a style that is  {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks into a style that is  American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [10]:
get_completion(prompt)

'"Well, I\'m quite frustrated about my blender lid flying off and splashing my kitchen walls with smoothie.  To add insult to injury, the warranty doesn\'t cover the cost of cleaning up the mess.  I could really use some assistance right away." \n'

In [11]:
# To do the same thing using LangChain

from langchain_ollama.chat_models import ChatOllama

In [12]:
chat = ChatOllama(model=model_name, temperature=0)
chat

ChatOllama(model='gemma2:2b-instruct-q4_1', temperature=0.0, _client=<ollama._client.Client object at 0x000001FEDED2F610>, _async_client=<ollama._client.AsyncClient object at 0x000001FEDED2FE50>)

In [13]:
template_str = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [14]:
from langchain.prompts import ChatPromptTemplate

# we use langchain chat prompt template to create a template that we can reuse many times
prompt_template = ChatPromptTemplate.from_template(template_str)

In [15]:
# we can see that this template has two types of inputs that are generated using our
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [16]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

customer_style = """American English \
in a calm and respectful tone
"""

In [17]:
# Now we can create customer messages that will generate the prompt using the template
# we have to fill the input variables 'style' and 'text' with our inputs
customer_messages = prompt_template.format_messages(style=customer_style, text=customer_email)
customer_messages

[HumanMessage(content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n")]

In [18]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n"


In [19]:
# On passe ce prompt généré par le template au modèle de chat créé avant
customer_response = chat.invoke(customer_messages)

In [20]:
customer_response.content

'"Well, I\'m quite upset about my blender lid flying off and making a mess on my kitchen walls with the smoothie.  To add insult to injury, the warranty doesn\'t cover the cost of cleaning up. I really could use some assistance right away." \n\n\nHere\'s what changed:\n\n* **Arrr** and **matey** were replaced with more formal language like "Well" and "I really could use".\n* The sentence structure was simplified for clarity.\n*  The tone is less aggressive and more polite, using phrases like "quite upset" instead of "fuming". \n\n\nThis version conveys the same message in a calmer and more respectful way. \n'

In [21]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [23]:
service_messages = prompt_template.format_messages(style=service_style_pirate, text=service_reply)
print(service_messages)

[HumanMessage(content="Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!\n```\n")]


In [24]:
service_response = chat.invoke(service_messages)

In [25]:
service_response

AIMessage(content="Ahoy, matey!  \n\nSo ye be wantin' a new blender, eh? Well, shiver me timbers, this here warranty don't cover no cleanin' expenses for yer kitchen. Seems ye forgot to put the lid on before givin' that ol' blender a whirl. A right mess it made, aye. \n\nBest of luck findin' another way to scrub up yer kitchen! Fair winds and following seas! \n", response_metadata={'model': 'gemma2:2b-instruct-q4_1', 'created_at': '2024-10-16T11:26:57.0914125Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 3929009600, 'load_duration': 41806500, 'prompt_eval_count': 82, 'prompt_eval_duration': 526956000, 'eval_count': 94, 'eval_duration': 3358761000}, id='run-cc6876d2-86c4-4cc4-8784-76a5397c772e-0', usage_metadata={'input_tokens': 82, 'output_tokens': 94, 'total_tokens': 176})

Les `parsers` LangChain permettent d'extraire et de formater la réponse en sortie d'un LLM afin de la réutiliser si besoin.  

In [26]:
# Exemple d'un format que nous souhaitons obtenir à partir d'une revue d'un produit
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [27]:
# On créé un template de revue où l'on spécifie au LLM exactement les informations à extraire de la revue faite par un client

customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [28]:
from langchain.prompts import ChatPromptTemplate

# we use langchain chat prompt template to create a template that we can reuse many times
prompt_template = ChatPromptTemplate.from_template(review_template)
prompt_template

ChatPromptTemplate(input_variables=['text'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'))])

In [29]:
# créons les messages que l'on va passer au LLM via le template

messages = prompt_template.format_messages(text=customer_review)
chat = ChatOllama(model=model_name, temperature=0)
res = chat.invoke(messages)

In [30]:
print(res.content)

```json
{
  "gift": "True",
  "delivery_days": "2",
  "price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features." 
}
```


In [31]:
# Même si la réponse a l'air bien formattée, on peut voir qu'il s'agit en réalité d'un long texte que l'on ne peut utiliser tel quel
# Il nous faut donc utiliser des parsers afin d'obtenir en réponse des objets directement utilisables
type(res.content)

str

In [32]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

# définir les schémas pour chaque valeur souhaitée
gift_schema = ResponseSchema(name='gift', description='"Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown."')

delivery_days_schema = ResponseSchema(name='delivery_days', description='How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.')

price_value_schema = ResponseSchema(name='price_value', description='Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.')

# les regrouper dans une liste
response_schemas = [gift_schema, delivery_days_schema, price_value_schema]

In [33]:
# créer le parser en utilisant la liste de schémas créée ci-dessus
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [34]:
# A partir du parser, on récupère les instructions de formattage
format_instructions = output_parser.get_format_instructions()

In [35]:
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"gift": string  // "Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown."\n\t"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.\n\t"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.\n}\n```'

In [36]:
# avec le même template que précédemment on ajoute les instructions de formattage de la réponse
review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}

{format_instructions}
"""

In [37]:
# on recréé ensuite le template de prompt et les messages avec ces nouvelles informations
prompt_template = ChatPromptTemplate.from_template(review_template)
messages = prompt_template.format_messages(text=customer_review, format_instructions=format_instructions)

In [38]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet form

In [39]:
res = chat.invoke(messages)

In [40]:
print(res.content)

```json
{
	"gift": "True",
	"delivery_days": "2",
	"price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```


In [41]:
type(res.content)

str

In [42]:
# On peut maintenant utiliser le parser créé afin de parser la réponse

output_dict = output_parser.parse(res.content)
type(output_dict)

dict

In [43]:
output_dict

{'gift': 'True',
 'delivery_days': '2',
 'price_value': "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."}

### <a id='memory'>2. Memory</a>

Les LLMs n'ont pas naturellement la capacité à "se rappeler" de l'historique d'une interaction; cela est dû à leurs limites en termes de tokens de contexte.  

Dans cette partie, nous verrons comment améliorer cet aspect en gardant en mémoire l'historique d'une interaction, en la remettant dans le prompt afin d'obtenir une conversation plus naturelle.

In [1]:
model_name = 'gemma2:2b-instruct-q4_1'

In [2]:
from langchain_ollama.chat_models import ChatOllama
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [3]:
llm = ChatOllama(model=model_name, temperature=0.0)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=False
)

C:\Users\Nelly\AppData\Local\Temp\ipykernel_16624\590950356.py:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  conversation = ConversationChain(


In [4]:
conversation.predict(input='Hi, my name is Audrey')

"Hi Audrey! It's nice to meet you. My name is Bard, an AI assistant created by Google.  What can I do for you today? 😊 \n"

In [5]:
conversation.predict(input='What is 1 + 1?')

"That's a simple one! 1 + 1 equals 2. 😊 \n"

In [6]:
conversation.predict(input='What is my name?')

"Audrey, it's nice to meet you too!  😊  You told me your name was Audrey.  What can I help you with today? \n"

In [7]:
# Modifier la valeur de verbose pour voir ce qui se passe sous le capot de LangChain

llm = ChatOllama(model=model_name, temperature=0.0)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

In [8]:
conversation.predict(input='Hi, my name is Audrey')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Audrey
AI:

> Finished chain.


"Hi Audrey! It's nice to meet you. My name is Bard, an AI assistant created by Google.  What can I do for you today? 😊 \n"

In [9]:
conversation.predict(input='What is 1 + 1?')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Audrey
AI: Hi Audrey! It's nice to meet you. My name is Bard, an AI assistant created by Google.  What can I do for you today? 😊 

Human: What is 1 + 1?
AI:

> Finished chain.


"That's a simple one! 1 + 1 equals 2. 😊 \n"

In [10]:
conversation.predict(input='What is my name?')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Audrey
AI: Hi Audrey! It's nice to meet you. My name is Bard, an AI assistant created by Google.  What can I do for you today? 😊 

Human: What is 1 + 1?
AI: That's a simple one! 1 + 1 equals 2. 😊 

Human: What is my name?
AI:

> Finished chain.


"Audrey, it's nice to meet you too!  😊  You told me your name was Audrey.  What can I help you with today? \n"

On voit qu'en utilisant le buffer de conversation, le modèle garde l'historique dans le prompt. Cela lui permet d'avoir une certaine "mémoire".

In [11]:
print(memory.buffer)

Human: Hi, my name is Audrey
AI: Hi Audrey! It's nice to meet you. My name is Bard, an AI assistant created by Google.  What can I do for you today? 😊 

Human: What is 1 + 1?
AI: That's a simple one! 1 + 1 equals 2. 😊 

Human: What is my name?
AI: Audrey, it's nice to meet you too!  😊  You told me your name was Audrey.  What can I help you with today? 



In [12]:
# On peut également ajouter manuellement des éléments au buffer
memory = ConversationBufferMemory()

memory.save_context({'input':'Hi'}, {'output':"What's up?"})

print(memory.buffer)

Human: Hi
AI: What's up?


In [13]:
memory.save_context({'input':'Nothing much, justing hanging'}, {'output':'Cool'})

print(memory.buffer)

Human: Hi
AI: What's up?
Human: Nothing much, justing hanging
AI: Cool


Essayons un autre type de buffer de conversation `qui s'étend sur une fenêtre qu'on définit`

In [14]:
from langchain.memory import ConversationBufferWindowMemory

In [15]:
# k = 1 pour garder seulement une fenêtre de contexte de 1
# (1 occurrence pour l'utilisateur et 1 occurrence pour le modèle)
memory = ConversationBufferWindowMemory(k=1)

In [16]:
memory.save_context({'input':'Hi'}, {'output':"What's up?"})
memory.save_context({'input':'Nothing much, justing hanging'}, {'output':'Cool'})

In [17]:
# avec ce buffer, le modèle garde bien en mémoire 1 seul élément pour chaque interlocuteur
print(memory.buffer)

Human: Nothing much, justing hanging
AI: Cool


Un autre type de buffer de conversation lié au `nombre de tokens`.  
Ce type de buffer est intéressant pour les cas où l'on travaille avec un modèle payant par exemple.

In [18]:
from langchain.memory import ConversationTokenBufferMemory

In [19]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)

In [20]:
memory.save_context({"input": "AI is what?!"}, {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"}, {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, {"output": "Charming!"})

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

g:\Utilisateurs\Nelly\Travail_Perso\python-envs\genai-test2\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Nelly\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

g:\Utilisateurs\Nelly\Travail_Perso\python-envs\genai-test2\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Un autre type de buffer de conversation se base sur un `un résumé`.  
Ce type de buffer utilise un llm pour résumer la conversation qui a eu lieu jusqu'à présent, et ce résumé est utilisé comme historique.

In [21]:
from langchain.memory import ConversationSummaryBufferMemory

In [22]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},{"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, {"output": f"{schedule}"})

### <a id='chains'>3. Chains</a>

La `chaîne`, qui correspond au bloc principal du framework LangChain, combine en général le LLM avec un prompt. En utilisant un tel bloc que l'on peut répéter, cela offre plusieurs possibilités intéressantes dans la création d'applications.  

Une possibilité intéressante des chaînes est qu'on peut l'utiliser sur plusieurs données en entrée simultanément.

In [5]:
model_name = 'gemma2:2b-instruct-q4_1'

In [1]:
# Importer des données avec pandas

from pathlib import Path
import pandas as pd

fichier = Path('../data/csv/chain-example.csv')
df = pd.read_csv(fichier)

df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...


In [3]:
# LLM chain basique

from langchain_ollama.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [6]:
llm = ChatOllama(model=model_name, temperature=0.9)

In [7]:
prompt = ChatPromptTemplate.from_template('What is the best name to describe a company that makes {product}?')

In [8]:
chain = LLMChain(llm=llm, prompt=prompt)

C:\Users\Nelly\AppData\Local\Temp\ipykernel_8980\1305865249.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [9]:
product = 'Queen Size Sheet Set'
chain.run(product)

C:\Users\Nelly\AppData\Local\Temp\ipykernel_8980\3772654808.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  chain.run(product)


"Here are some names that could work for a Queen-size sheet set company, categorized by style:\n\n**Classic & Elegant:**\n\n* **Royal Rest** \n* **Grand Sheets**\n* **Serene Slumbers**\n* **The Velvet Nest**\n* **Queen's Comfort**\n* **Majestic Sleep**\n\n**Modern & Playful:**\n\n* **Sheet Kings/Queens** (fun, bold)\n* **Dreamworthy**\n* **The Pillow Post** \n* **Snuggle Co.**\n* **Cloud Nine Sheets**\n\n**Descriptive & Direct:**\n\n* **Queen Size Bedding Company**\n* **Luxury Queen Sheets**\n* **Premium Queen Sets**\n* **Deep Sleep Collection**\n\n\n**Tips for Choosing the Right Name:**\n\n* **Target audience:**  Who are you selling to? Consider their age, lifestyle, and preferences. \n* **Brand personality:** Do you want your name to be playful, luxurious, or straightforward?\n* **Memorability:** Is the name easy to remember and pronounce?\n* **Availability:** Check if the name is available as a domain name and social media handle.\n\n\n**Beyond Words: The Importance of Logo & Brandi

In [10]:
# Simple Sequential chain : combine exactement 1 input et 1 output de chaque chaîne
# dans l'ordre
from langchain.chains import SimpleSequentialChain

In [19]:
first_prompt = ChatPromptTemplate.from_template('What is the best name to describe a company that makes {product}? Give me just one name.')

chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [22]:
second_prompt = ChatPromptTemplate.from_template('Write a 20 words description for the following company:{company_name}')

chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [23]:
overall_simple_chain = SimpleSequentialChain(
    chains = [chain_one, chain_two],
    verbose = True
)

In [24]:
overall_simple_chain.invoke(product)



> Entering new SimpleSequentialChain chain...
**Slumber Haven** 

Slumber Haven provides cozy, comfortable bedding and sleep solutions for ultimate relaxation. 


> Finished chain.


{'input': 'Queen Size Sheet Set',
 'output': 'Slumber Haven provides cozy, comfortable bedding and sleep solutions for ultimate relaxation. \n'}

In [25]:
# Lorsqu'il y a plus d'un input et output, on utilise le Sequential chain
from langchain.chains import SequentialChain

In [26]:
first_prompt = ChatPromptTemplate.from_template(
    """Translate the following review to English:
    \n\n{review}
    """
    )

chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key='english_review')

In [27]:
second_prompt = ChatPromptTemplate.from_template(
    """Can you summarize the following review in 1 sentence:
    \n\n{english_review}
    """
    )

chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key='summary')

In [28]:
third_prompt = ChatPromptTemplate.from_template(
    """In what language is the following review:
    \n\n{review}
    """
    )

chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key='language')

In [33]:
fourth_prompt = ChatPromptTemplate.from_template(
    """Write a follow up response to the following summary in the specified language:
    \n\nSummary: {summary}\n\nLanguage: {language}
    """
    )

chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key='followup_msg')

In [34]:
overall_chain = SequentialChain(
    chains = [chain_one, chain_two, chain_three, chain_four],
    input_variables = ['review'],
    output_variables = ['english_review', 'summary', 'followup_msg'],
    verbose = True
)

In [36]:
review = df.Review[5]
overall_chain.invoke(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?",
 'english_review': "I find the taste mediocre. The foam doesn't hold, it's strange. I buy the same ones at the store and the taste is much better...\n\nIs this old stock or counterfeit!? \n",
 'summary': 'This reviewer found the taste of the product subpar compared to their store-bought option and suspects potential issues with authenticity. \n',
 'followup_msg': '## Réponse à l\'analyse :\n\n**"La saveur du produit s\'avère en dépit des attentes, se rapprochant peu de la qualité d’un produit acheté dans une épicerie, et suspecte un potentiel problème d\'authenticité."**\n\n---\n\nLet me explain:\n\n* **"This reviewer found the taste of the product subpar compared to their store-bought option"**: This translates directly and accurately conveys the essence of the summary. \n* **"se rapprochant peu de la qualit

In [37]:
# Pour des cas plus complexes, il est intéressant d'utiliser un router qui peut choisir
# à quel autre chaîne passer le résultat


physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [38]:
# on crée des informations sur les différents prompts selon leur sujet de prédilection
# ces informations seront passées au routeur pour aider dans sa décision
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [45]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

In [41]:
destination_chains = {}

for p in prompt_infos:
    name = p['name']
    prompt_template = p['prompt_template']
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [42]:
# valeurs par défaut au cas où le router n'a pas assez d'informations pour choisir
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [43]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [46]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template = router_template,
    input_variables=['input'],
    output_parser=RouterOutputParser()
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [47]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True
)

C:\Users\Nelly\AppData\Local\Temp\ipykernel_8980\2813769297.py:1: LangChainDeprecationWarning: Use RunnableLambda to select from multiple prompt templates. See example in API reference: https://api.python.langchain.com/en/latest/chains/langchain.chains.router.multi_prompt.MultiPromptChain.html
  chain = MultiPromptChain(


In [48]:
chain.invoke('What is black body radiation?')



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


{'input': 'What is black body radiation?',
 'text': "Great question!  Blackbody radiation,  is the light emitted by an object that absorbs all incident electromagnetic radiation, regardless of frequency or angle. Think of it as the most efficient radiator possible. \n\nIt doesn't matter if it's a star burning in space, a hot pan on your stove, or even a tiny particle - if an object is heated to a high enough temperature, it emits blackbody radiation. \n\n\n**The Key Players:**\n* **Temperature (T):** This determines the peak wavelength of emitted light and is directly linked to the object's internal energy.\n* **Electromagnetic Radiation:**  Blackbody radiation consists of all frequencies of electromagnetic waves - but we see its impact most clearly as visible light.\n\n**Why Is It Important?**\n\nBlackbody radiation has immense implications for:\n* **Understanding stars**: The spectrum of light emitted by stars is a direct consequence of their temperature and composition.\n* **The bla

In [49]:
chain.invoke('What is 2 + 2?')



> Entering new MultiPromptChain chain...
math: {'input': 'What is 2 + 2?'}
> Finished chain.


{'input': 'What is 2 + 2?',
 'text': 'Let\'s break it down:\n\n* **Component 1:** We are adding two numbers together (2 and 2).  \n* **Component 2:** This involves the operation of addition, represented by the "+" symbol.  \n* **Solution:** To find the solution to this problem, we add the first number to the second number (2 + 2 = 4).\n\n**Therefore, 2 + 2 equals 4.** \n\n\nLet me know if you have any other math problems you\'d like help with! 😄  \n'}

On voit bien que chaque question est bien routée vers la bonne chaîne pour obtenir une réponse à la question posée.

### <a id='question-and-answer'>4. Question and answer</a>

Cette fonctionnalité de LangChain permet de construire des applications qui combinent les LLMs avec des documents sur lesquels ils n'ont pas été entraînés.  

Cela offre de grandes possibilités car les LLMs en deviennent ainsi plus flexibles.

In [1]:
model_name = 'gemma2:2b-instruct-q4_1'

In [2]:
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_ollama.llms import OllamaLLM
from langchain.chains import RetrievalQA
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch

from IPython.display import display, Markdown
from pathlib import Path

In [3]:
# On crée un loader LangChain à partir du fichier de données
fichier = Path('../data/csv-txt/langchain/retrieval-outdoor-clothing.csv')
loader = CSVLoader(file_path=fichier, encoding='utf8')

In [4]:
# pour nous aider à créer un index de vecteur plus facilment
from langchain.indexes import VectorstoreIndexCreator

In [33]:
llm = OllamaLLM(model=model_name, temperature=0.5)

In [5]:
# création de l'index
embeddings = OllamaEmbeddings(model=model_name)
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings
    ).from_loaders([loader])

g:\Utilisateurs\Nelly\Travail_Perso\python-envs\genai-test2\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [42]:
query = "Please list every shirt that have sun protection \
in a markdown table, and summarize each one in less than 20 words. The name \
attribute must mention the word 'shirt' for it to be a shirt. If a shirt does have \
sun protection, the description will mention something similar to 'UPF 50+'."

In [43]:
res = index.query(query, llm=llm)

In [44]:
display(Markdown(res))

| Name | Summary |
|---|---|
| Men's Plaid Tropic Shirt, Short-Sleeve | Lightweight, wrinkle-free shirt with UPF 50+ sun protection. |  




On peut refaire cet exemple en procédant par étapes pour plus de compréhension.

In [3]:
from langchain.document_loaders import CSVLoader

# 1. On crée un loader LangChain à partir du fichier de données
fichier = Path('../data/csv/retrieval-outdoor-clothing.csv')
loader = CSVLoader(file_path=fichier, encoding='utf8')

In [4]:
# 2. On charge ensuite les documents à partir du loader
docs = loader.load()

In [5]:
print(docs[0])

page_content=': 0
name: Women's Campside Oxfords
description: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. 

Size & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. 

Specs: Approx. weight: 1 lb.1 oz. per pair. 

Construction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. 

Questions? Please contact us for any inquiries.' metadata={'source': '..\\data\\csv\\retrieval-outdoor-clothing.csv', 'row': 0}


In [ ]:
# 3. On découpe les éléments en "chunks" afin de les insérer dans le vector store

# PS: dans le cas actuel, les documents dont nous disposons sont individuellement
# petits comme on peut le voir ci-dessus, donc pas besoin de les découper. 
# On peut donc directement créer les embeddings sur ces documents.

In [6]:
# 4. On crée les embeddings

from langchain_ollama import OllamaEmbeddings
embeddings = OllamaEmbeddings(model=model_name)

In [7]:
# vérifier ce que fait l'embedding
test_embed = embeddings.embed_query('Hi my name is Audrey')

In [8]:
print(len(test_embed))
print(test_embed)

2304
[-0.009170529, 0.002634769, -0.005046889, 0.009464011, 0.005023745, 0.008658773, -0.010745666, 0.0044666496, -0.0056383368, -0.0018385494, -0.0118973935, -0.011663985, -0.010653571, -0.0071752984, 0.0016451138, -0.0022024151, -0.004184519, -0.0023072183, -0.029984558, 0.004578143, 0.0019222953, -0.023707282, -0.013256571, -0.0020219907, -0.0014535794, -0.0034331062, -0.010205655, 0.00637563, -0.00927417, -0.004122926, -0.00010651525, -0.006243533, -0.010386501, -0.0026935895, 0.009630466, 0.017238561, -0.0010671235, 0.0036802157, 0.0015628256, -0.0119682755, 0.0029880374, 0.0030912233, 0.015683398, 0.0094699515, 0.003024296, -0.017271144, -0.0024301768, 0.0009829041, 0.0041862023, 0.0010873448, 0.0028968055, 0.015280737, -0.0005752692, -0.016067047, -0.018051513, 0.0035680167, 0.0077957776, 0.0015329652, 0.001622359, 0.0016017782, -0.020544639, 0.041803535, -0.0136966575, 0.0051219063, 0.008351156, -0.0017880438, -0.0059062014, -0.0001971106, -0.0012041304, 0.0104490165, 0.0084074

On voit que cette variable test contient l'embedding de notre simple phrase; il s'agit d'un vecteur de chiffres qui représente la version 'numérique' de notre phrase.  

Ce vecteur est de `dimension 3584` (colonnes).

In [9]:
# création de l'embedding pour tous nos documents, on le stocke ici en mémoire plutôt
# qu'une vrai base de données de vecteurs pour les besoins de l'exemple

db = DocArrayInMemorySearch.from_documents(documents=docs, embedding=embeddings)

g:\Utilisateurs\Nelly\Travail_Perso\python-envs\genai-test2\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [22]:
# 5. Utiliser la base de données pour effectuer une recherche

query = 'Please suggest a shirt with sunblocking'
srch = db.similarity_search(query)

In [23]:
len(srch)

4

In [24]:
print(srch[3])

page_content=': 709
name: Sunrise Tee
description: Stay cool, comfortable and dry on the hottest days in our women's UV-protective button down shirt. The lightweight, high-performance fabric wicks away moisture and dries quickly.

Size & Fit
Slightly Fitted: Softly shapes the body. Falls at hip.

Why We Love It
Our lightest hot-weather shirt lets you beat the heat. Originally designed for fishing, it's also a great choice for travel thanks to its wrinkle-free fabric and built-in sun protection with a rating of UPF 50+.

Fabric & Care
Lightweight performance synthetic wicks moisture, resists wrinkles and dries fast. Shell: 71% nylon, 29% polyester. Cape lining: 100% polyester. Machine wash and dry.

Additional Features
Built-in SunSmart™ UPF 50+ rated – the highest rated sun protection possible. The high-performance fabric keeps you cool and comfortable by wicking perspiration away. Smoother buttons, low-profile pockets and side shaping for a flattering fit. Front and back cape venting.

Pour pouvoir utiliser cela afin de faire un question-answer avec nos documents, nous devons créer un retriever.  

Il s'agit d'une interface qui peut utiliser n'importe quelle méthode sous-jacente et capable de répondre à une question donnée en renvoyant des documents.  
La recherche de vecteurs et les embeddings correspondent à l'une des méthodes possibles.

In [40]:
# 6. Créer un retriever
retriever = db.as_retriever()
llm = OllamaLLM(model=model_name, temperature=0.9)

In [31]:
# retriever manuel : on joint tous les documents 'à la main'
q_srch = "".join([docs[i].page_content for i in range(len(docs))])
res = llm.invoke(f"{q_srch} Question: Please list all your \
shirts with sun protection in a markdown table and summarize each one. Do not mix up \
shirt with other product types.")

In [30]:
display(Markdown(res))

## Shirts with Sun Protection

| Name | Description | Summary |
|---|---|---|
| Women's Stop Flies Hoodie |  Uses No Fly Zone Technology to repel ticks, mosquitoes, flies and other biting insects, while also providing UPF 40+ sun protection. Slightly fitted for a soft shape, falls at the hip for added coverage. Breathable and wicks away moisture. | Offers bug protection and sun protection in one garment with a comfortable fit.  | 


Let me know if you'd like to explore other product categories or have any more questions! 


In [41]:
# Retriever utilisant une chaîne

qa_stuff = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=retriever,
    verbose=True
)

In [42]:
query = 'Please list all your \
shirts with sun protection in a markdown table and summarize each one. Do not mix up \
shirt with other product types.'

qa_stuff.invoke(query)



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Please list all your shirts with sun protection in a markdown table and summarize each one. Do not mix up shirt with other product types.',
 'result': "| Name | Description | Summary |\n|---|---|---|\n| Ultimate Two-Layer Polo | This polo shirt is crafted with premium long-staple 100% cotton, honeycomb-knit and garment-washed for softness. It's comfortable enough for the tennis court yet tailored for everyday wear.  | Offers comfort and durability for a variety of settings, ideal for businesses looking to add logos to clothing |\n| Men's Plaid Tropic Shirt, Short-Sleeve | This shirt is designed for extended travel and offers UPF 50+ protection against harmful UV rays, keeping you cool and dry in hot weather. It features wrinkle-free fabric and ventilation for a comfortable experience.  | Designed specifically for sun protection and warmth in warmer climates.| \n"}

In [43]:
display(Markdown(res))

## Shirts with Sun Protection

| Name | Description | Summary |
|---|---|---|
| Women's Stop Flies Hoodie |  Uses No Fly Zone Technology to repel ticks, mosquitoes, flies and other biting insects, while also providing UPF 40+ sun protection. Slightly fitted for a soft shape, falls at the hip for added coverage. Breathable fabric with wrist and cuff that won't ride up and thumbhole for added coverage. |  Offers bug protection and sun protection in one garment. Lightweight, comfortable, and stylish. | 


### <a id='evaluation'>5. Evaluation</a>

Les modèles de langages peuvent parfois proposer des réponses pas tout à fait exactes lorsqu'ils sont interrogés.  
Pour construire des applicatiosns robustes, il nous faut trouver un moyen d'évaluer les réponses : cela peut se faire manuellement (à l'oeil), mais également en utilisant un LLM.

In [1]:
model_name = 'gemma2:2b-instruct-q4_1'

In [2]:
from langchain.chains import RetrievalQA
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch
from pathlib import Path

In [3]:
# On crée un loader LangChain à partir du fichier de données
fichier = Path('../data/csv-txt/langchain/retrieval-outdoor-clothing.csv')
loader = CSVLoader(file_path=fichier, encoding='utf8')
data = loader.load()

In [4]:
embeddings = OllamaEmbeddings(model=model_name)
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings
    ).from_loaders([loader])

g:\Utilisateurs\Nelly\Travail_Perso\python-envs\genai-test2\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [5]:
llm = ChatOllama(model=model_name, temperature=0)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=index.vectorstore.as_retriever(),
    verbose=True,
    chain_type_kwargs={'document_separator':'"<<<<>>>>>"'}
)

On peut maintenant utiliser des exemples de données sur lesquels on souhaite effectuer une évaluation

In [9]:
print(data[10])
print('-----')
print(data[11])

page_content=': 10
name: Cozy Comfort Pullover Set, Stripe
description: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.

Size & Fit
- Pants are Favorite Fit: Sits lower on the waist.
- Relaxed Fit: Our most generous fit sits farthest from the body.

Fabric & Care
- In the softest blend of 63% polyester, 35% rayon and 2% spandex.

Additional Features
- Relaxed fit top with raglan sleeves and rounded hem.
- Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg.

Imported.' metadata={'source': '..\\data\\csv-txt\\langchain\\retrieval-outdoor-clothing.csv', 'row': 10}
-----
page_content=': 11
name: Ultra-Lofty 850 Stretch Down Hooded Jacket
description: This technical stretch down jacket from our DownTek collection is sure to keep you warm and comfortable with its full-stretch construction providing exceptiona

In [27]:
# Exemples de paires question-réponse manuels pour les documents 10 et 11 vus plus haut

examples = [
    {
        'query': 'Do the Cozy Comfort Pullover Set have side pockets?',
        'answer': 'yes'
    },
    {
        'query': 'What collection is the Ultra-Lofty 850 Stretch Down Hooded Jacket from?',
        'answer': 'The DownTek collection'
    }
]

Les exemples créés plus haut impliquent de choisir manuellement nos exemples (documents 10 et 11), de créer des questions et de vérifier les bonnes réponses à ces questions. Ensuite, on fournit ces éléments au modèle pour pouvoir l'évaluer.  

Cette technique est relativement sûre, mais ne permet pas de passer à l'échelle.  

Si l'on veut aller plus loin, une possibilité est `d'utiliser une autre chaîne avec un LLM en tant qu'évaluateur`. 

In [31]:
from langchain.evaluation.qa import QAGenerateChain, QAEvalChain

In [11]:
# QAGenerateChain utilise les documents qu'on lui donne en entrée pour créer des paires de question-réponse
example_generated_chain = QAGenerateChain.from_llm(llm)

In [13]:
generated_examples = example_generated_chain.apply_and_parse([{'doc': t} for t in data[:5]])

g:\Utilisateurs\Nelly\Travail_Perso\python-envs\genai-test2\Lib\site-packages\langchain\chains\llm.py:367: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [14]:
print(generated_examples[0])

{'qa_pairs': {'query': "What is the approximate weight of a pair of Women's Campside Oxfords?", 'answer': "The approximate weight of a pair of Women's Campside Oxfords is 1 lb.1 oz. per pair. "}}


In [28]:
# Ajoutons ces exemples générés à ceux créés manuellement

examples += [d['qa_pairs'] for d in generated_examples]

In [19]:
import langchain
langchain.debug = True

In [20]:
qa.run(examples[0]['query'])

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Do the Cozy Comfort Pullover Set have side pockets?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Do the Cozy Comfort Pullover Set have side pockets?",
  "context": "- Stretch-paneled “shove it” pocket provides quick access to trail essentials.\"<<<<>>>>>\": 69\nname: Women's Camo Mountain Fleece Vest\ndescription: Designed for unbeatable warmth and plush comfort, our camouflage heavyweight Sherpa-Fleece Vest looks great from mountain peak to base lodge. Why We Love It: We turned to our archives for inspiration when designing this extraplush sherpa fleece. It's the perfect layer for cold days on the mountain, gathering around the campfire, or running into town. \r\n\r\nFabric & Care: Thick and cozy 300-weight sherpa fleece pr

'The provided text does not mention a "Cozy Comfort Pullover Set". \n'

In [21]:
langchain.debug = False

In [30]:
predictions = qa.apply(examples)



> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


In [32]:
eval_chain = QAEvalChain.from_llm(llm)

In [35]:
graded_outputs = eval_chain.evaluate(examples, predictions)

In [38]:
graded_outputs

[{'results': 'GRADE: INCORRECT \n'},
 {'results': 'GRADE: CORRECT \n'},
 {'results': 'GRADE: INCORRECT \n'},
 {'results': 'GRADE: INCORRECT \n'},
 {'results': 'GRADE: INCORRECT \n'},
 {'results': 'GRADE: INCORRECT \n'},
 {'results': 'GRADE: INCORRECT \n'}]

In [39]:
# Check des résultats

for i, example in enumerate(examples):
    print(f'Exemple {i} -------> : ')
    print('Question: ', predictions[i]['query'])
    print('Réponse attendue: ', predictions[i]['answer'])
    print('Réponse prédite: ', predictions[i]['result'])
    print('Note prédite: ', graded_outputs[i]['results'])

Exemple 0 -------> : 
Question:  Do the Cozy Comfort Pullover Set have side pockets?
Réponse attendue:  yes
Réponse prédite:  The provided text does not mention a "Cozy Comfort Pullover Set". 

Note prédite:  GRADE: INCORRECT 

Exemple 1 -------> : 
Question:  What collection is the Ultra-Lofty 850 Stretch Down Hooded Jacket from?
Réponse attendue:  The DownTek collection
Réponse prédite:  This text does not contain information about the Ultra-Lofty 850 Stretch Down Hooded Jacket or any collections it might be part of. 

Note prédite:  GRADE: CORRECT 

Exemple 2 -------> : 
Question:  What is the approximate weight of a pair of Women's Campside Oxfords?
Réponse attendue:  The approximate weight of a pair of Women's Campside Oxfords is 1 lb.1 oz. per pair. 
Réponse prédite:  This document does not contain information about Women's Campside Oxfords. 

Note prédite:  GRADE: INCORRECT 

Exemple 3 -------> : 
Question:  What is the construction of the Recycled Waterhog dog mat? 
Réponse att

### <a id='agents'>6. Agents</a>

Les agents sont des outils basés sur les LLMs, mais que l'on équipe afin qu'ils puissent interagir de façon autonome avec d'autres outils : on parle alors de 'moteur de raisonnement', et il s'agit de la meilleure manière d'utiliser les LLMs.  

In [1]:
model_name = 'gemma2:2b-instruct-q4_1'

In [14]:
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain_experimental.agents.agent_toolkits.python.base import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_experimental.utilities import PythonREPL

In [15]:
llm = ChatOllama(model = model_name, temperature=0)

In [20]:
# on charge les outils 'llm-math' (une chaîne qui combine llm + calculatrice), et 'wikipedia' pour requêter la base Wikipédia
tools = load_tools(['llm-math', 'wikipedia'], llm=llm)

In [21]:
# Initialiser un agent

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True
)

C:\Users\Nelly\AppData\Local\Temp\ipykernel_15364\235751029.py:3: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  agent = initialize_agent(


In [ ]:
agent("What is the 25% of 300?")

In [ ]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
agent(question) 

In [27]:
# Créer un agent avec Python

agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [28]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]]

In [29]:
agent.run(f'''Sort these customers by last name and then first name and print the output: {customer_list}''')

C:\Users\Nelly\AppData\Local\Temp\ipykernel_15364\340590393.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  agent.run(f'''Sort these customers by last name and then first name and print the output: {customer_list}''')




> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


Thought: I need to sort the list of customers by last name and then first name. 
Action: Python_REPL
Action Input: 
```python
customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]

customers.sort(key=lambda x: (x[1], x[0]))
print(customers) 
```
Observation: [['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]

Thought:Thought: I now know the final answer.
Final Answer: [['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']] 


> Finished chain.


"[['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]"

In [30]:
import langchain
langchain.debug=True
agent.run(f'''Sort these customers by last name and then first name and print the output: {customer_list}''')
langchain.debug=False

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatOllama] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nIf you get an

In [43]:
# Créer des outils personnalisés pour interagir avec les agents
# Exemple : création d'un outil qui nous indique la date

from langchain.agents import tool
from datetime import date

@tool
def current_date(text: str) -> str:
    '''
    Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function.
    '''
    return str(date.today())

In [44]:
agent = initialize_agent(
    tools + [current_date],
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True
)

In [ ]:
agent.run('What is the current date?')